In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import jax
from jax import numpy as jnp
import optax
from functools import partial
import sim
import optuna
import matplotlib.pyplot as plt

In [2]:
jax.devices()

[cuda(id=0)]

In [3]:
!optuna-dashboard sqlite:///mango4x4.db

Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [08/Feb/2024 09:41:19] "GET / HTTP/1.1" 302 0
127.0.0.1 - - [08/Feb/2024 09:41:19] "GET /api/studies HTTP/1.1" 200 133
127.0.0.1 - - [08/Feb/2024 09:41:21] "GET /api/meta HTTP/1.1" 200 32
/home/davide_sartor/.conda/envs/dl_env/lib/python3.10/site-packages/optuna/study/_study_summary.py:115: FutureWarning: `system_attrs` has been deprecated in v3.1.0. The removal of this feature is currently scheduled for v5.0.0, but this schedule is subject to change. See https://github.com/optuna/optuna/releases/tag/v3.1.0.
  warnings.warn(
/home/davide_sartor/.conda/envs/dl_env/lib/python3.10/site-packages/optuna/study/_study_summary.py:115: FutureWarning: `system_attrs` has been deprecated in v3.1.0. The removal of this feature is currently scheduled for v5.0.0, but this schedule is subject to change. See https://github.com/optuna/optuna/releases/tag/v3.1.0.
  warnings.warn(
127.0.0.1 - - [08/Feb/2024 09:41:21] "GET /api/studies/

In [3]:
def objective(trial):
    seed = 42
    map_size = 4
    max_interactions = 100_000
    map_scale = int(jnp.log2(map_size))

    # hyperparams
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    rollout_length = 2 * map_size**2
    n_rollouts = max_interactions // rollout_length
    train_iter = 8

    # setup annealing schedules
    annealing_durations = [
        int(train_iter * n_rollouts * trial.suggest_float(f"annealing_{i}", 0.1, 1.0 / map_scale))
        for i in range(map_scale)
    ]
    annealing_starts = jnp.cumsum(jnp.array([0] + annealing_durations[:-1]))
    annealing_schedules = [
        optax.linear_schedule(1.0, 0.01, dur, start)
        for dur, start in zip(annealing_durations, annealing_starts)
    ]

    @jax.jit
    def annealing_fn(step):
        return jnp.array([schedule(step) for schedule in annealing_schedules])

    run_simulation = partial(
        sim.run_mango_simulation,
        map_size=map_size,
        n_rollouts=n_rollouts,
        rollout_length=rollout_length,
        train_iter=train_iter,
        lr=lr,
        annealing_fn=annealing_fn,
    )
    rng_key = jax.random.PRNGKey(seed)
    env, dql_state, results = jax.vmap(run_simulation)(jax.random.split(rng_key, 8))

    best = results.eval_reward[:, :, :, 0, -1].max(axis=0)
    worst = results.eval_reward[:, :, :, 0, -1].min(axis=0)
    accuracy = (best.mean() + worst.mean())/2

    return accuracy


study = optuna.create_study(
    study_name="mango",
    storage="sqlite:///mango4x4.db",
    load_if_exists=True,
    direction="maximize"
)

study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-02-08 07:37:09,027] A new study created in RDB with name: mango


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

[I 2024-02-08 07:38:17,796] Trial 0 finished with value: 0.03718499839305878 and parameters: {'lr': 0.00014236230525882483, 'annealing_0': 0.34457509950697773, 'annealing_1': 0.47744952834674026}. Best is trial 0 with value: 0.03718499839305878.


  0%|          | 0/3125 [00:00<?, ?it/s]

...

[I 2024-02-08 09:10:25,561] Trial 97 finished with value: 0.05612500011920929 and parameters: {'lr': 0.0002446247376571047, 'annealing_0': 0.1196810474627796, 'annealing_1': 0.3399644369063081}. Best is trial 64 with value: 0.06030000001192093.


  0%|          | 0/3125 [00:00<?, ?it/s]

[I 2024-02-08 09:11:20,641] Trial 98 finished with value: 0.05447499826550484 and parameters: {'lr': 0.00014811424719140612, 'annealing_0': 0.14987887719743048, 'annealing_1': 0.32592121209535746}. Best is trial 64 with value: 0.06030000001192093.


  0%|          | 0/3125 [00:00<?, ?it/s]

[I 2024-02-08 09:12:17,577] Trial 99 finished with value: 0.04742499813437462 and parameters: {'lr': 0.00030333931288757627, 'annealing_0': 0.13638393633695456, 'annealing_1': 0.2816103518986375}. Best is trial 64 with value: 0.06030000001192093.
